## Importing 

In [1]:
# importing
import psutil
import pandas as pd
import numpy as np
# from pandas_profiling import ProfileReport
from matplotlib import pyplot
import matplotlib.pyplot as plt
plt.figure(num=None, figsize=(10, 8), dpi=80, facecolor='w', edgecolor='k')
import seaborn as sns; sns.set() # for plot styling
from pandas.plotting import scatter_matrix
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing

import seaborn as sns
import warnings
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import csv
warnings.filterwarnings('ignore')

## Read file of October

In [2]:

svmem = psutil.virtual_memory()
# print (svmem.available)
PATH = r"archive/2019-Oct.csv"
df_sample = pd.read_csv(PATH, nrows=10)
df_sample_size = df_sample.memory_usage(index=True).sum()
# print (df_sample_size)
# print (df_sample)
# define a chunksize that would occupy a maximum of 1Gb
# we divide by 10 because we have selected 10 lines in our df_sample
my_chunk = (1000000000 / df_sample_size)/10
my_chunk = int(my_chunk//1) # we get the integer part
# print (my_chunk)
# create the iterator
iter_csv = pd.read_csv(
    PATH,
    iterator=True,
    chunksize=my_chunk)
# concatenate according to a filter to our result dataframe
df_oct = pd.concat(
    [chunk[chunk['price']>0]
    for chunk in iter_csv])
# print (df_oct)

In [3]:
# drop the missing values in user_session since there aren't too many values
df_oct = df_oct.drop(df_oct[df_oct.user_session.isnull()].index)

In [4]:
# convert event_time to datetime
df_oct['event_time'] = pd.to_datetime(df_oct['event_time'], 
                                       format='%Y-%m-%d %H:%M:%S %Z')

In [5]:
# drop the columns
# df_oct = df_oct.drop(['product_id','category_id','category_code','brand','user_session'])
df_oct.drop(['product_id','category_id','category_code','brand','user_session'], axis='columns', inplace=True)

## Read file of November

In [7]:

svmem = psutil.virtual_memory()
# print (svmem.available)
PATH = r"archive/2019-Nov.csv"
df_sample = pd.read_csv(PATH, nrows=10)
df_sample_size = df_sample.memory_usage(index=True).sum()
# print (df_sample_size)
# print (df_sample)
# define a chunksize that would occupy a maximum of 1Gb
# we divide by 10 because we have selected 10 lines in our df_sample
my_chunk = (1000000000 / df_sample_size)/10
my_chunk = int(my_chunk//1) # we get the integer part
# print (my_chunk)
# create the iterator
iter_csv = pd.read_csv(
    PATH,
    iterator=True,
    chunksize=my_chunk)
# concatenate according to a filter to our result dataframe
df_nov = pd.concat(
    [chunk[chunk['price']>0]
    for chunk in iter_csv])
# print (df_nov)

In [8]:
# drop the missing values in user_session since there aren't too many values
df_nov = df_nov.drop(df_nov[df_nov.user_session.isnull()].index)

In [9]:
# convert event_time to datetime
df_nov['event_time'] = pd.to_datetime(df_nov['event_time'], 
                                       format='%Y-%m-%d %H:%M:%S %Z')

In [10]:
# drop the columns
# df_oct = df_oct.drop(['product_id','category_id','category_code','brand','user_session'])
df_nov.drop(['product_id','category_id','category_code','brand','user_session'], axis='columns', inplace=True)

## Read file of December

In [145]:

svmem = psutil.virtual_memory()
# print (svmem.available)
PATH = r"archive/2019-Dec.csv"
df_sample = pd.read_csv(PATH, nrows=10)
df_sample_size = df_sample.memory_usage(index=True).sum()
# print (df_sample_size)
# print (df_sample)
# define a chunksize that would occupy a maximum of 1Gb
# we divide by 10 because we have selected 10 lines in our df_sample
my_chunk = (1000000000 / df_sample_size)/10
my_chunk = int(my_chunk//1) # we get the integer part
# print (my_chunk)
# create the iterator
iter_csv = pd.read_csv(
    PATH,
    iterator=True,
    chunksize=my_chunk)
# concatenate according to a filter to our result dataframe
df_dec = pd.concat(
    [chunk[chunk['price']>0]
    for chunk in iter_csv])
# print (df_dec)

In [146]:
# drop the missing values in user_session since there aren't too many values
df_dec = df_dec.drop(df_dec[df_dec.user_session.isnull()].index)

In [147]:
# convert event_time to datetime
df_dec['event_time'] = pd.to_datetime(df_dec['event_time'], 
                                       format='%Y-%m-%d %H:%M:%S %Z')

In [148]:
# drop the columns
# df_oct = df_oct.drop(['product_id','category_id','category_code','brand','user_session'])
new_dec = df_dec
new_dec.drop(['product_id','category_id','category_code','brand','user_session'], axis='columns', inplace=True)

In [149]:
# get the max Invoice date
max_date = max(df_dec['event_time'])
# max_date

In [150]:
# Get how many days does the Invoice have that is done from the max Invoice
new_dec.insert(4, "Recency", max_date-new_dec['event_time'], True) 
new_dec['Recency'] = new_dec['Recency'].dt.days

In [151]:
# new_dec.drop(['event_time'], axis='columns', inplace=True)

In [171]:
view_dec = new_dec.loc[new_dec.event_type=='view'].groupby(
   ['user_id'],as_index=False
).agg(
    {
         'event_type':"count",    # Sum duration per group
         'Recency': "max",  # get the count of networks
         'price': sum  # get the first date per group
    }
)
view_dec.rename(columns = {'event_type':'view'}, inplace = True)
# view_dec

In [172]:
cart_dec = new_dec.loc[new_dec.event_type=='cart'].groupby(
   ['user_id'],as_index=False
).agg(
    {
         'event_type':"count",    # Sum duration per group
         'Recency': "max",  # get the count of networks
         'price': sum  # get the first date per group
    }
)
cart_dec.rename(columns = {'event_type':'cart'}, inplace = True)
# cart_dec

In [162]:
rem_dec = new_dec.loc[new_dec.event_type=='remove_from_cart'].groupby(
   ['user_id'],as_index=False
).agg(
    {
         'event_type':"count",    # Sum duration per group
         'Recency': "max",  # get the count of networks
         'price': sum  # get the first date per group
    }
)
rem_dec.rename(columns = {'event_type':'remove_from_cart'}, inplace = True)
# rem_dec

In [159]:
pur_dec = new_dec.loc[new_dec.event_type=='purchase'].groupby(
   ['user_id'],as_index=False
).agg(
    {
         'event_type':"count",   
         'Recency': "max", 
         'price': sum
    }
)
pur_dec.rename(columns = {'event_type':'purchase'}, inplace = True)
# pur_dec

In [207]:
cust_dec = view_dec.merge(cart_dec, left_on='user_id', right_on='user_id', how='left')
cust_dec = cust_dec.drop(['Recency_y','price_y','Recency_x','price_x'],1)
cust_dec['cart'].fillna('0', inplace=True)
# cust_dec

,user_id,view,cart
0,1180452,1,0
1,2963072,17,7
2,4661182,3,2
3,4891613,1,0
4,6217356,1,0
...,...,...,...
357792,595413843,1,0
357793,595413976,1,0
357794,595414210,1,0
357795,595414257,1,0


In [208]:
cust_dec = cust_dec.merge(rem_dec, left_on='user_id', right_on='user_id', how='left')
cust_dec = cust_dec.drop(['Recency','price'],1)
cust_dec['remove_from_cart'].fillna('0', inplace=True)
# cust_dec

,user_id,view,cart,remove_from_cart
0,1180452,1,0,0
1,2963072,17,7,0
2,4661182,3,2,2
3,4891613,1,0,0
4,6217356,1,0,0
...,...,...,...,...
357792,595413843,1,0,0
357793,595413976,1,0,0
357794,595414210,1,0,0
357795,595414257,1,0,0


In [209]:
cust_dec = cust_dec.merge(pur_dec, left_on='user_id', right_on='user_id', how='left')
# cust_dec = cust_dec.drop(['Recency','price'],1)
cust_dec['purchase'].fillna('0', inplace=True)
cust_dec['Recency'].fillna('0', inplace=True)
cust_dec['price'].fillna('0', inplace=True)
# cust_dec

,user_id,view,cart,remove_from_cart,purchase,Recency,price
0,1180452,1,0,0,0,0,0
1,2963072,17,7,0,0,0,0
2,4661182,3,2,2,0,0,0
3,4891613,1,0,0,0,0,0
4,6217356,1,0,0,0,0,0
...,...,...,...,...,...,...,...
357792,595413843,1,0,0,0,0,0
357793,595413976,1,0,0,0,0,0
357794,595414210,1,0,0,0,0,0
357795,595414257,1,0,0,0,0,0
